Sentiment Analysis with Transformer-based model
       
       Full Analysis approach

        Read reviews from SQL Server Databse,

        Clean text

        Run a transformer sentiment classifier (uses cardiffnlp/twitter-roberta-base-sentiment which outputs Negative/Neutral/Positive),

        Evaluate against rating-derived weak labels (so you can see how model aligns with your ratings),

        Write results to CSV and (optionally) back to SQL.

Install Libraries:

pip install pandas sqlalchemy pyodbc transformers torch tqdm scikit-learn



End-to-end sentiment analysis for reviews stored in SQL Server.

Model: cardiffnlp/twitter-roberta-base-sentiment (negative/neutral/positive)

In [1]:
# Import Libraties

import re
import os
import math
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sqlalchemy import create_engine
import urllib

c:\Users\farhan\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ---------------------
# Defining Values
# ---------------------

SQL_SERVER = "DESKTOP-NN982MH\SQLEXPRESS"         
SQL_DB     = "PortfolioProject_MarketingAnalytics"           
SQL_UID    = ""                        
SQL_PWD    = ""                       
USE_TRUSTED_CONNECTION = True          # True -> windows auth (trusted); False -> use UID/PWD

TABLE_NAME = "dbo.customer_reviews"    
OUTPUT_CSV = "customer_reviews_with_sentiment.csv"
BATCH_SIZE = 32                        
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"  

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\farhan\AppData\Local\Temp\ipykernel_5760\2373366630.py:1: SyntaxWarning: invalid escape sequence '\S'
  SQL_SERVER = "DESKTOP-NN982MH\SQLEXPRESS"


In [3]:
# ---------------------
# SQL connection
# ---------------------

def get_sqlalchemy_engine():
    if USE_TRUSTED_CONNECTION:
        conn_str = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={SQL_SERVER};DATABASE={SQL_DB};Trusted_Connection=yes;"
        )
    else:
        conn_str = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={SQL_SERVER};DATABASE={SQL_DB};UID={SQL_UID};PWD={SQL_PWD};"
        )
    params = urllib.parse.quote_plus(conn_str)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)
    return engine


In [4]:
# ---------------------
# 1) Load data from SQL
# ---------------------
engine = get_sqlalchemy_engine()
sql = f"SELECT CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM {TABLE_NAME}"
print("Loading data from SQL...")
df = pd.read_sql(sql, engine)
print(f"Loaded {len(df)} rows.")

Loading data from SQL...
Loaded 1363 rows.


In [5]:
# ---------------------
# 2) Basic text cleaning
# ---------------------
def clean_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    s = s.strip()
    # remove URLs
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    # remove HTML tags
    s = re.sub(r"<.*?>", " ", s)
    # replace multiple spaces
    s = re.sub(r"\s+", " ", s)
    # optional: remove excessive punctuation
    s = s.replace("\n", " ").replace("\r", " ")
    return s.strip()

df['ReviewText'] = df['ReviewText'].fillna("").astype(str)
df['clean_text'] = df['ReviewText'].apply(clean_text)

# drop rows with empty text (optional)
df = df[df['clean_text'].str.len() > 0].reset_index(drop=True)

print(f"After dropping empty reviews: {len(df)} rows.")

After dropping empty reviews: 1363 rows.


In [6]:
# ---------------------
# 3) Prepare model pipeline
# ---------------------

print("Loading model and tokenizer...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Create HF pipeline; set device=0 for GPU, -1 for CPU
device = 0 if (os.environ.get("CUDA_VISIBLE_DEVICES") or False) else -1
# If you have GPU but environment variable isn't set, set device=0 manually above.
sentiment_pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

# label mapping fallback (some models return LABEL_0 style labels)
_map = {0: "negative", 1: "neutral", 2: "positive"}

def interpret_label(label_str: str):
    lbl = label_str.lower()
    if lbl in ("positive", "negative", "neutral"):
        return lbl.capitalize()
    # coerce LABEL_n
    if lbl.startswith("label_"):
        idx = int(lbl.split("_")[1])
        return _map.get(idx, "neutral").capitalize()
    # fallback
    return lbl.capitalize()

Loading model and tokenizer...


c:\Users\farhan\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\farhan\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not 

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [7]:
# ---------------------
# 4) Batch inference
# ---------------------

texts = df['clean_text'].tolist()
pred_labels = []
pred_scores = []

print("Running inference in batches...")

for i in tqdm(range(0, len(texts), BATCH_SIZE)):
    batch = texts[i:i+BATCH_SIZE]
    # pipeline will return list of dicts, each with 'label' and 'score'
    results = sentiment_pipe(batch, truncation=True, max_length=512)
    for res in results:
        label = interpret_label(res.get('label', "neutral"))
        score = float(res.get('score', 0.0))
        pred_labels.append(label)
        pred_scores.append(score)

Running inference in batches...


100%|██████████| 43/43 [00:47<00:00,  1.11s/it]


In [8]:
# Add results to dataframe

df['Sentiment_Pred'] = pred_labels
df['Sentiment_Score'] = pred_scores

In [9]:
# ---------------------
# 5) Quick evaluation vs rating (weak labels)
# ---------------------
def rating_to_sentiment(r):
    try:
        r = float(r)
    except:
        return "Neutral"
    if r >= 4.0:
        return "Positive"
    elif r == 3.0:
        return "Neutral"
    else:
        return "Negative"

df['Rating_Label'] = df['Rating'].apply(rating_to_sentiment)

print("\nClassification report (model vs rating-derived labels):")
print(classification_report(df['Rating_Label'], df['Sentiment_Pred'], digits=4))

cm = confusion_matrix(df['Rating_Label'], df['Sentiment_Pred'], labels=["Positive","Neutral","Negative"])
cm_df = pd.DataFrame(cm, index=["True_Pos","True_Neu","True_Neg"], columns=["Pred_Pos","Pred_Neu","Pred_Neg"])
print("\nConfusion matrix (rows=true, cols=pred):")
print(cm_df)


Classification report (model vs rating-derived labels):
              precision    recall  f1-score   support

    Negative     0.6302    0.9142    0.7461       233
     Neutral     0.8312    0.2207    0.3488       290
    Positive     0.8861    1.0000    0.9396       840

    accuracy                         0.8195      1363
   macro avg     0.7825    0.7116    0.6781      1363
weighted avg     0.8306    0.8195    0.7808      1363


Confusion matrix (rows=true, cols=pred):
          Pred_Pos  Pred_Neu  Pred_Neg
True_Pos       840         0         0
True_Neu       101        64       125
True_Neg         7        13       213


1. Overall accuracy

        accuracy = 0.8195  (~82%)

        The model agrees with the rating-derived sentiment labels 82% of the time. That’s strong overall.

2. Class-wise metrics

        🔹 Positive

                Precision: 0.8861 → 89% of predicted positives are correct.

                Recall: 1.0000 → Model caught all true positives.

                F1: 0.9396 → Excellent.
                ✅ Interpretation: The model is highly reliable at detecting Positive reviews.


        🔹 Negative

                Precision: 0.6302 → 63% of predicted negatives are correct (so ~37% false positives).

                Recall: 0.9142 → The model catches most negative reviews, but sometimes mislabels other reviews as negative.

                F1: 0.7461 → Decent but not perfect.
                ⚠️ Interpretation: The model tends to over-predict negatives.

        🔹 Neutral

                Precision: 0.8312 → When it predicts Neutral, it’s right most of the time.

                Recall: 0.2207 → But it misses most actual neutral reviews (only 22% detected).

                F1: 0.3488 → Weak overall.
                ⚠️ Interpretation: The model struggles to identify Neutral sentiment (likely because ratings → sentiment mapping is noisy, and human-written neutral reviews are often subtle).

3. Confusion Matrix

              Pred_Pos  Pred_Neu  Pred_Neg
True_Pos         840        0        0
True_Neu         101       64      125
True_Neg           7       13      213


        True_Pos (840) → All 840 actual positive reviews were predicted as Positive (perfect recall).

        True_Neg (233) → Most negatives correctly predicted (213), but a few got confused as Neutral (13) or Positive (7).

        True_Neu (290) → Huge problem:

        101 mislabeled as Positive,

        125 mislabeled as Negative,

        Only 64 correctly predicted Neutral.

👉 Takeaway: The model basically collapses Neutral into Positive/Negative.

In [13]:
# ---------------------
# 6) Save outputs
# ---------------------

df_out = df[['CustomerID','ProductID','ReviewDate','Rating','ReviewText',
             'clean_text','Rating_Label','Sentiment_Pred','Sentiment_Score']]

df_out.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved predictions to {OUTPUT_CSV}")

# Save classification report to CSV
report_dict = classification_report(df['Rating_Label'], df['Sentiment_Pred'], 
                                    digits=4, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv("classification_report.csv", index=True)
print("Saved classification_report.csv")

# Save confusion matrix to CSV
cm = confusion_matrix(df['Rating_Label'], df['Sentiment_Pred'], 
                      labels=["Positive","Neutral","Negative"])
cm_df = pd.DataFrame(cm, 
                     index=["True_Pos","True_Neu","True_Neg"], 
                     columns=["Pred_Pos","Pred_Neu","Pred_Neg"])
cm_df.to_csv("confusion_matrix.csv", index=True)
print("Saved confusion_matrix.csv")



Saved predictions to customer_reviews_with_sentiment.csv
Saved classification_report.csv
Saved confusion_matrix.csv
